## Exploring Air Quality over the Southwest United States due to the California Wildfires

NOAA's GOES-16 (EAST) Advanced Baseline Imager (ABI) sensor highlights the spatial and temporal evolution of the aerosol optical depth (AOD) over the southwest United States due the California wildfires between August 15 and 25, 2020. Majority of the wildfires were caused by cloud-to-ground lightning strikes ([NASA](https://www.nasa.gov/feature/goddard/2020/nasa-s-suomi-npp-satellite-highlights-california-wildfires-at-night)).
In this work, the first image from each hour were concatenated and converted into Graphics Interchange Format (GIF) for animation, although this senor has a temporal resoltuion of 5 minutes. Detailed explanation on the acquisition of these AOD imageries from the Amazon's S3 bucket can be accessed and cloned through this GitHub repository [NOAA-GOES-16-ABI-AOD](https://github.com/EngIyasu/NOAA-GOES-16-ABI-AOD). 

### Required libraries

In [1]:
from netCDF4 import Dataset
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap, cm
import numpy as np
import requests
import boto3
import datetime 
from IPython.display import Image
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.animation as animation
from IPython.display import HTML, display

%matplotlib inline

### Accessing the images through the AWS CLI credentials

In [2]:
s3_client = boto3.client('s3',
                         aws_access_key_id = 'your_aws_access_key_id',
                         aws_secret_access_key = 'your_aws_secret_access_key') 

In [3]:
def get_s3_keys(bucket, s3_client, prefix = ''):
    """
    Generate the keys in an S3 bucket.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch keys that start with this prefix (optional).
    """
    
    kwargs = {'Bucket': bucket}

    if isinstance(prefix, str):
        kwargs['Prefix'] = prefix

    while True:
        resp = s3_client.list_objects_v2(**kwargs)
        for obj in resp['Contents']:
            key = obj['Key']
            if key.startswith(prefix):
                yield key

        try:
            kwargs['ContinuationToken'] = resp['NextContinuationToken']
        except KeyError:
            break

### Mapping the images

In [4]:
bucket_name = 'noaa-goes16'
product_name = 'ABI-L2-AODC' #Advanced Baseline Imager Level 2 Aerosol Optical Depth CONUS
year = 2020

# Hours list
list_1 = [0,1]
list_2 = list(range(15,24))
com_list = list_1 + list_2

for day_of_year in range(228,239): 
    for hour in com_list:
        
        keys = get_s3_keys(bucket_name,
                   s3_client,
                   prefix = f'{product_name}/{year}/{day_of_year:03.0f}/{hour:02.0f}/OR_{product_name}')
        
        keys_combined = []
        for i in keys:
            keys_combined.append(i)
                            
        fig, ax = plt.subplots(1,1, figsize=(19,20))#(width,height)
        #fig.subplots_adjust(hspace=0, wspace=0.1)    
        single_file = keys_combined[0]
        file_name = single_file.split('/')[-1].split('.')[0]
    
        ## Search for the Scan start in the file name
        start = (file_name[file_name.find("_s")+2:file_name.find("_e")])
    
        ## Converting from julian day to dd-mm-yyyy
        year = int(start[0:4])
        dayjulian = int(start[4:7]) - 1 # Subtract 1 because the year starts at "0"
        dayconventional = datetime.datetime(year,1,1) + datetime.timedelta(dayjulian) # Convert from julian to conventional
        date = dayconventional.strftime('%d-%b-%Y')  # Format the date according to the strftime directives
        time = start[7:9] + ":" + start[9:11] + ":" + start[11:13] + " UTC" # Time of the Start of the Scan
        
        ## Response from AWS S3
        resp = requests.get(f'https://{bucket_name}.s3.amazonaws.com/{single_file}')
    
        ## Open the file using the NetCDF4 library
        nc = Dataset(file_name, memory = resp.content)
    
        ## Parameters required to naviage data points on ABI fixed grid
        x_rad_1d = nc.variables['x'][:]
        y_rad_1d = nc.variables['y'][:]
        x_rad,y_rad = np.meshgrid(x_rad_1d,y_rad_1d)
        AOD_550nm = nc.variables['AOD'][:]
    
        ## GOES-R projection info and retrieving relevant constants
        variable_req = nc.variables['goes_imager_projection']
        lambda_0 = (variable_req.longitude_of_projection_origin)*(np.pi/180)
        H = variable_req.perspective_point_height + variable_req.semi_major_axis
        r_eq = variable_req.semi_major_axis
        r_pol = variable_req.semi_minor_axis
    
        # close file when finished
        nc.close()
        nc = None
    
        ## Navigate from N/S elevstion sngle (y) and E/W scanning angle (x) to geodetic latitue and longitude
        a_var = np.power((np.sin(x_rad)),2) + (np.power((np.cos(x_rad)),2)*(np.power((np.cos(y_rad)),2)+(((r_eq*r_eq)/(r_pol*r_pol))*np.power(np.sin(y_rad),2))))
        b_var = (-2.0*H)*(np.cos(x_rad)*np.cos(y_rad))
        c_var = (H**2)-(r_eq**2)

        r_s = (-1.0*b_var - np.sqrt((b_var**2)-(4.0*a_var*c_var)))/(2.0*a_var) #Distance from the satellite to point P in the ground surface

        s_x = r_s*np.cos(x_rad)*np.cos(y_rad)
        s_y = -1 * r_s * np.sin(x_rad)
        s_z = r_s*np.cos(x_rad)*np.sin(y_rad)
    
        ## Latitude and longitude projection for plotting data on traditional lat/lon maps    
        lat = (np.arctan(((r_eq*r_eq)/(r_pol*r_pol))*((s_z/np.sqrt(((H-s_x)*(H-s_x))+(s_y*s_y))))))*(180.0/np.pi)
        lon = (lambda_0 - np.arctan(s_y/(H-s_x)))*(180.0/np.pi)
    
        data_bounds = np.where(AOD_550nm.data!=65535)
        bbox = [np.min(lon[data_bounds]),
                np.min(lat[data_bounds]),
                np.max(lon[data_bounds]),
                np.max(lat[data_bounds])] 
            
        n_add = 0 # for zooming in and out
        m = Basemap(ax = ax,llcrnrlon=-126-n_add,llcrnrlat=31-n_add,urcrnrlon=-105+n_add,\
                urcrnrlat=48+n_add,resolution='i', projection='cyl')
    
        m.shadedrelief()
        m.drawcoastlines(color='black',linewidth=1.5)
        m.drawcountries(color='black',linewidth=1.5)
        m.drawstates(color='black',linewidth=1.5)
                    
        vmin = 0
        vmax = 2.0 #np.nanmax(AOD_550nm[:])
        set_1 = m.pcolormesh(lon.data, lat.data, AOD_550nm, latlon=True,zorder=999, cmap = "jet",vmin=vmin, vmax=vmax)
        title_name = date + ', ' + time
        ax.set_title(title_name,fontsize=20) 
    
        del AOD_550nm
    
        parallels = np.arange(15,60,5.); # make latitude lines ever 5 degrees from 30N-50N
        meridians = np.arange(-125,-60,5.); 
        m.drawparallels(parallels,linewidth=0.7,labels=[1,0,0,0],fontsize=18,dashes=[6, 18]);
        m.drawmeridians(meridians,linewidth=0.7,labels=[0,0,0,1],fontsize=18,dashes=[6, 18]);
                       
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("bottom", size="4%", pad="4%")
        ticks = np.linspace(vmin, vmax, 8)
        cb = fig.colorbar(set_1, ax=ax,extend='both', cax=cax, orientation="horizontal",ticks=ticks)
        cb.ax.tick_params(labelsize=18)
        
        save_name = 'output_images/GOES_ABI_AOD_' + date + time + '.png'
        plt.savefig(save_name, dpi=150, bbox_inches = 'tight')
        #plt.show() 
        plt.close(fig='all')

/Users/iyasueibed/anaconda3/envs/geo_spatial_env/lib/python3.7/site-packages/ipykernel_launcher.py:64: RuntimeWarning: invalid value encountered in sqrt
/Users/iyasueibed/anaconda3/envs/geo_spatial_env/lib/python3.7/site-packages/ipykernel_launcher.py:71: RuntimeWarning: invalid value encountered in sqrt


### Converting Output Images into GIF

In [5]:
!convert -delay 30 ./output_images/GOES_ABI_AOD_*.png ./animation/GOES_ABI_AOD_Southwest_20200815_20200825.gif

## Resize the GIF file
!gifsicle ./animation/GOES_ABI_AOD_Southwest_20200815_20200825.gif --resize 1208x1027 > ./animation/GOES_ABI_AOD_Southwest_20200815_20200825_resize.gif

gifsicle: warning: huge GIF, conserving memory (processing may take a while)
gifsicle: warning: too many colors, using local colormaps
  (You may want to try ‘--colors 256’.)


### Displaying the GIF Images

In [6]:
display(HTML("<img src='./animation/GOES_ABI_AOD_Southwest_20200815_20200825_resize.gif' />"))